In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import pyranges as pr
import scikit_posthocs as sp
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import scipy
from numpy import log10
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
import statannot
pd.set_option('mode.chained_assignment',  None)

## Importing Data and variable declaration

In [ ]:
### batch-corrected DESeq2(timecours - LRT method) results

file = './source_data/DEmRNA/group1_padj_score.txt'
DEG_G1 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group2_padj_score.txt'
DEG_G2 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group3_padj_score.txt'
DEG_G3 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group4_padj_score.txt'
DEG_G4 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group5_padj_score.txt'
DEG_G5 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group6_padj_score.txt'
DEG_G6 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/group7_padj_score.txt'
DEG_G7 = pd.read_csv(file, sep = '\t', header = 0)


### batch-corrected DESeq2 results with WGCNA filtered

file = './source_data/DEmRNA/WGCNA_group2_padj_score.txt'
PT1 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/WGCNA_group3_padj_score.txt'
PT2 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/WGCNA_group4_padj_score.txt'
PT3 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DEmRNA/WGCNA_group6_padj_score.txt'
PT4 = pd.read_csv(file, sep = '\t', header = 0)

WDEG_ALL = pd.concat([PT1,PT2,PT3,PT4],axis=0)

### batch-corrected DESeq2(pairwise - WARD method) results (paired analysis of G1-G2, G2-G3, G3-G4)

file = './source_data/DEmRNA/G1-G2_upDEG.txt'
DEG_G1_G2_up = pd.read_csv(file, sep = '\t', header= 0)
file = './source_data/DEmRNA/G1-G2_downDEG.txt'
DEG_G1_G2_down = pd.read_csv(file, sep = '\t', header= 0)

file = './source_data/DEmRNA/G2-G3_upDEG.txt'
DEG_G2_G3_up = pd.read_csv(file, sep = '\t', header= 0)
file = './source_data/DEmRNA/G2-G3_downDEG.txt'
DEG_G2_G3_down = pd.read_csv(file, sep = '\t', header= 0)

file = './source_data/DEmRNA/G3-G4_upDEG.txt'
DEG_G3_G4_up = pd.read_csv(file, sep = '\t', header= 0)
file = './source_data/DEmRNA/G3-G4_downDEG.txt'
DEG_G3_G4_down = pd.read_csv(file, sep = '\t', header= 0)



### Kallisto - isoformswitchanalyzeR proccessed DTU results (paired analysis of G1-G2, G2-G3, G3-G4)

file = './source_data/Isoformswitchanalyzer/G1_G2_isoformswitchlist_sig.txt'
DTU_G1_G2_all = pd.read_csv(file, sep = '\t', names=['isoform_id','gene_id','gene','condition_1','condition_2','isoform_switch_q_value','dIF','IF1','IF2'])
DTU_G1_G2_up = DTU_G1_G2_all[(DTU_G1_G2_all['isoform_switch_q_value'] < 0.05) & (DTU_G1_G2_all['dIF'] > 0.1)]
DTU_G1_G2_down = DTU_G1_G2_all[(DTU_G1_G2_all['isoform_switch_q_value'] < 0.05) & (DTU_G1_G2_all['dIF'] < -0.1)]
DTU_G1_G2 = DTU_G1_G2_all[(DTU_G1_G2_all['isoform_switch_q_value'] < 0.05) & ((DTU_G1_G2_all['dIF'] > 0.1) | (DTU_G1_G2_all['dIF'] < -0.1))]

file = './source_data/Isoformswitchanalyzer/G2_G3_isoformswitchlist_sig.txt'
DTU_G2_G3_all = pd.read_csv(file, sep = '\t', names=['isoform_id','gene_id','gene','condition_1','condition_2','isoform_switch_q_value','dIF','IF1','IF2'])
DTU_G2_G3_up = DTU_G2_G3_all[(DTU_G2_G3_all['isoform_switch_q_value'] < 0.05) & (DTU_G2_G3_all['dIF'] > 0.1)]
DTU_G2_G3_down = DTU_G2_G3_all[(DTU_G2_G3_all['isoform_switch_q_value'] < 0.05) & (DTU_G2_G3_all['dIF'] < -0.1)]
DTU_G2_G3 = DTU_G2_G3_all[(DTU_G2_G3_all['isoform_switch_q_value'] < 0.05) & ((DTU_G2_G3_all['dIF'] > 0.1) | (DTU_G2_G3_all['dIF'] < -0.1))]

file = './source_data/Isoformswitchanalyzer/G3_G4_isoformswitchlist_sig.txt'
DTU_G3_G4_all = pd.read_csv(file, sep = '\t', names=['isoform_id','gene_id','gene','condition_1','condition_2','isoform_switch_q_value','dIF','IF1','IF2'])
DTU_G3_G4_up = DTU_G3_G4_all[(DTU_G3_G4_all['isoform_switch_q_value'] < 0.05) & (DTU_G3_G4_all['dIF'] > 0.1)]
DTU_G3_G4_down = DTU_G3_G4_all[(DTU_G3_G4_all['isoform_switch_q_value'] < 0.05) & (DTU_G3_G4_all['dIF'] < -0.1)]
DTU_G3_G4 = DTU_G3_G4_all[(DTU_G3_G4_all['isoform_switch_q_value'] < 0.05) & ((DTU_G3_G4_all['dIF'] > 0.1) | (DTU_G3_G4_all['dIF'] < -0.1))]


##### Transcriptcount_CPM
file = './source_data/kallisto/TrptCPM_kallisto_mfa.txt'
TrptCPM = pd.read_csv(file, sep = '\t', header = 0)
TrptCPM = TrptCPM.set_index('Transcript')

#### Genecount_CPM_macaca fascicularis (bat_corection 데이터로 변경된 )
file = './source_data/batch_correction/batch_cor_GeneCPM_mfa.txt'
GeneCPM = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM = GeneCPM.set_index('gene_chr')

#### Genecount_CPM_ macaca mulatta
file = './source_data/count_result/GeneCPM_mml.txt'
GeneCPM_mml = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM_mml = GeneCPM_mml.set_index('gene_chr')


#### Genecount_rawcount_batchcorrection_macaca fascicularis (bat_corection 데이터로 변경된 )
file = './source_data/batch_correction/batch_cor_GeneCount_mfa.txt'
GeneRawcount_batchCo_mfa = pd.read_csv(file, sep = '\t', header = 0)
GeneRawcount_batchCo_mfa= GeneRawcount_batchCo_mfa.set_index('gene_chr')


#### Deseq2_LRT_median ratios nomalization_DEG_macaca fascicularis (bat_corection 데이터로 변경된 )
file = './source_data/DEmRNA/batch_cor_Gene_LRT_normalized_count_TC_mfa.txt'
DEG_norm_count_mfa = pd.read_csv(file, sep = '\t', header = 0)


#### cancer gene list
file = './source_data/annotation_files/cancer_gene_census.csv'
cancer_gene = pd.read_csv(file,encoding='latin-1')
cancer_gene['Role'] = cancer_gene['Role in Cancer'].astype(str)
cancer_gene['role_list'] = cancer_gene.Role.str.split(', ')
cancer_gene['oncogene'] = cancer_gene.apply(lambda x: 'Y' if 'oncogene' in x['role_list'] else 'N', axis=1)
cancer_gene['TSG'] = cancer_gene.apply(lambda x: 'Y' if 'TSG' in x['role_list'] else 'N', axis=1)
cancer_gene['fusion'] = cancer_gene.apply(lambda x: 'Y' if 'fusion' in x['role_list'] else 'N', axis=1)

cancer_gene = cancer_gene[['Gene Symbol','oncogene','TSG','fusion']]
cancer_gene = cancer_gene.rename(columns={'Gene Symbol':'gene'})


#### genAge gene list
file = './source_data/annotation_files/genage_human.csv'
genage_gene  = pd.read_csv(file)
genage_gene = genage_gene[['symbol','name']]
genage_gene = genage_gene.rename(columns={'symbol':'gene'})


#### cellAge gene list
file = './source_data/annotation_files/cellAge.csv'
cellage_gene  = pd.read_csv(file, sep = ';', header = 0)
cellage_gene = cellage_gene[['gene_name','senescence_effect']]
cellage_gene = cellage_gene.rename(columns={'gene_name':'gene'})
cellage_gene['effect'] = cellage_gene.apply(lambda x: 'H' if x['senescence_effect'] == 'Inhibits' else 'D' if x['senescence_effect'] == 'Induces' else 'U', axis=1)
cellage_gene = cellage_gene[['gene','effect']]

##### innate gene list
file = './source_data/annotation_files/InnateDB_genes.txt'
innate_gene = pd.read_csv(file, sep = '\t', header = 0)
innate_gene = innate_gene[innate_gene['species'] == 'Homo sapiens']
innate_gene = innate_gene[['name']]
innate_gene['innate'] = 'Y'
innate_gene = innate_gene.rename(columns={'name':'gene'})


#### generate annot gene 
genage_gene['aging'] = 'Y'
genage_gene = genage_gene[['gene','aging']]
annot_gene = pd.merge(genage_gene,cancer_gene,how='outer',on=['gene'])
annot_gene = pd.merge(annot_gene,cellage_gene,how='outer',on=['gene'])
annot_gene =annot_gene.replace(np.nan, 'N') 


#### TIMER (bat_corection 데이터로 변경된 )
file = './source_data/TIMER/human_TIMER_GROUP_DLBC_TPM.csv'
TIMER_group = pd.read_csv(file, sep = ',', header = 0)

file = './source_data/TIMER/human_TIMER_SAMPLE_DLBC_TPM.csv'
TIMER_sample = pd.read_csv(file, sep = ',', header = 0)



#### hubgene
file = './source_data/hubgene/hubgene_20.txt'
hubgenes = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/hubgene/hubgene_cancer.txt'
hubgenes_cancer = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/hubgene/hubgene_605.txt'
hubgenes_605 = pd.read_csv(file, sep = '\t', header = 0)


#### immune_marker
immune_mark = pd.read_excel('./source_data/annotation_files/Immune_cell_marker_gene_rheMac.xlsx',header=0,skiprows=1)


#### GeneCPM_condition
file = './source_data/reference/mml_condition.txt'
condition = pd.read_csv(file, sep = ' ', names=['sample','age','id','sex'])

In [ ]:
def file_parsing(data):

    data['gene'] = data.index.str.split('_').str[0]

    data_melt = pd.melt(data, id_vars=['gene'])
    conditionlist = [data_melt.variable == 'A_FST',
                     data_melt.variable == 'B_FST',
                     data_melt.variable == 'A_SND',
                     data_melt.variable == 'B_SND',
                     data_melt.variable == 'A_TRD',
                     data_melt.variable == 'B_TRD',
                 
                     data_melt.variable == 'C_FST',
                     data_melt.variable == 'D_FST',
                     data_melt.variable == 'C_SND',
                     data_melt.variable == 'D_SND',
                     data_melt.variable == 'C_TRD',
                     data_melt.variable == 'D_TRD',
                 
                     data_melt.variable == 'E_FST',
                     data_melt.variable == 'F_FST',
                     data_melt.variable == 'E_SND',
                     data_melt.variable == 'F_SND',
                     data_melt.variable == 'E_TRD',
                     data_melt.variable == 'F_TRD',
                 
                     data_melt.variable == 'G_FST',
                     data_melt.variable == 'H_FST',
                     data_melt.variable == 'G_SND',
                     data_melt.variable == 'H_SND',
                     data_melt.variable == 'G_TRD',
                     data_melt.variable == 'H_TRD']
    choicelist = [3,3,4,4,5,5,7,7,8,8,9,9,12,12,13,13,14,14,16,16,17,17,18,18]
    data_melt['age'] = np.select(conditionlist,choicelist, default='Not Specified')


    conditionlist = [data_melt.variable.str.split('_').str[0] == 'A',
                     data_melt.variable.str.split('_').str[0] == 'B',
                     data_melt.variable.str.split('_').str[0] == 'C',
                     data_melt.variable.str.split('_').str[0] == 'D',
                     data_melt.variable.str.split('_').str[0] == 'E',
                     data_melt.variable.str.split('_').str[0] == 'F',
                     data_melt.variable.str.split('_').str[0] == 'G',
                     data_melt.variable.str.split('_').str[0] == 'H']
    choicelist = ['G1','G1','G2','G2','G3','G3','G4','G4']
    data_melt['Group'] = np.select(conditionlist,choicelist, default='Not Specified')

    data_melt['age'] = pd.to_numeric(data_melt['age'])
    
    return data_melt

In [ ]:
DEG_ALL = pd.DataFrame()
DEG_A_ALL = pd.DataFrame()
DTU_ALL = pd.DataFrame()
for n,i in enumerate(['G1_G2_up','G1_G2_down','G2_G3_up','G2_G3_down','G3_G4_up','G3_G4_down']):
    
    ## DEG
    globals()['DEG_{}'.format(i)]['G'] = i
    globals()['DEG_{}'.format(i)]['Group'] = i.split('_')[0] + i.split('_')[1]
    DEG_ALL = pd.concat([DEG_ALL,globals()['DEG_{}'.format(i)]], axis=0)
    
    globals()['DEG_C_{}'.format(i)] = pd.merge(globals()['DEG_{}'.format(i)],annot_gene,how='inner',on=['gene'],indicator=True)
    globals()['DEG_C_{}'.format(i)]['G'] = '{}_{}'.format(str(n),i)
    DEG_A_ALL = pd.concat([DEG_A_ALL,globals()['DEG_C_{}'.format(i)]], axis=0)
    
    ## DTU
    globals()['DTU_{}'.format(i)]['G'] = i
    globals()['DTU_{}'.format(i)]['Group'] = i.split('_')[0] + i.split('_')[1]
    globals()['DTU_{}'.format(i)]['isoform'] = globals()['DTU_{}'.format(i)].isoform_id.str.replace('ENSMFAT000000','E::',case=False)
    globals()['DTU_{}'.format(i)]['gene'] = globals()['DTU_{}'.format(i)]['gene']+'_'+globals()['DTU_{}'.format(i)]['isoform']
    DTU_ALL = pd.concat([DTU_ALL,globals()['DTU_{}'.format(i)]], axis=0) 
    
DEG_ALL = DEG_ALL.dropna(axis=0)
DEG_A_ALL = DEG_A_ALL.dropna(axis=0)
DTU_ALL = DTU_ALL.dropna(axis=0)

## Figure S1c ; Figure S2b

In [ ]:
from upsetplot import UpSet, from_indicators, generate_counts
import pylab as plot
import gpar

def upsetplot(df,df_name):
    
    df = df.dropna(axis=0)
    df = df[~(df.gene.str.startswith('LOC'))]
    DF = df[['G','gene']]
    DF = DF.set_index('gene')

    all_G = DF['G'].unique().tolist()
    all_G.reverse()
    all_gene = DF.index.unique().tolist()
    
    A = DF[DF['G']==all_G[0]]
    B = DF[DF['G']==all_G[1]]
    C = DF[DF['G']==all_G[2]]
    D = DF[DF['G']==all_G[3]]
    E = DF[DF['G']==all_G[4]]
    F = DF[DF['G']==all_G[5]]
    
    TOT = pd.concat([A,B,C,D,E,F],axis=1)
    TOT.columns=all_G

    TOT = TOT.notnull()

    TOT_G = TOT.groupby(all_G).size()
    
    if df_name == 'DEG':
        #DEG
        DEG_upset = UpSet(TOT_G,show_counts=True, show_percentages=False,orientation='horizontal',sort_categories_by=None,facecolor='black')
        DEG_upset.style_subsets(present=['G1_G2_up', 'G2_G3_up', 'G3_G4_down'],facecolor='orange',label='UP-UP-DOWN')
        DEG_upset.style_subsets(present=['G1_G2_up', 'G2_G3_down', 'G3_G4_up'],facecolor='brown',label='UP-DOWN-UP')
        DEG_upset.style_subsets(present=['G1_G2_down', 'G2_G3_up', 'G3_G4_down'],facecolor='blue',label='DOWN-UP-DOWN')

        params = {'legend.fontsize':8}
        with plt.rc_context(params):
            DEG_upset.plot()      
        
    if df_name == 'DTU':
        #DTU
        DTU_upset = UpSet(TOT_G,show_counts=True, show_percentages=False,orientation='horizontal',sort_categories_by=None,facecolor='black')
        DTU_upset.style_subsets(present=['G2_G3_down', 'G3_G4_up'],facecolor='orange',label='G2G3_DOWN-G3G4_UP')
        DTU_upset.style_subsets(present=['G2_G3_up', 'G3_G4_down'],facecolor='brown',label='G2G3_UP-G3G4_DOWN')
        DTU_upset.style_subsets(present=['G1_G2_down', 'G2_G3_up'],facecolor='blue',label='G1G2_DOWN-G2G3_UP')
        DTU_upset.style_subsets(present=['G1_G2_up', 'G2_G3_down'],facecolor='green',label='G1G2_UP-G2G3_DOWN')
    
        params = {'legend.fontsize':6}
        with plt.rc_context(params):
            DTU_upset.plot()
        
        
print(upsetplot(DEG_ALL,'DEG'))
print(upsetplot(DTU_ALL,'DTU'))                         

## Figure 1c

In [ ]:
DEG_up = pd.concat([DEG_G1_G2_up, DEG_G2_G3_up, DEG_G3_G4_up],axis=0)
DEG_down = pd.concat([DEG_G1_G2_down, DEG_G2_G3_down, DEG_G3_G4_down],axis=0)
DEG_UP_DOWN = pd.concat([DEG_up,DEG_down],axis=0)

DTU_up = pd.concat([DTU_G1_G2_up, DTU_G2_G3_up, DTU_G3_G4_up],axis=0)
DTU_down = pd.concat([DTU_G1_G2_down, DTU_G2_G3_down, DTU_G3_G4_down],axis=0)
DTU_UP_DOWN = pd.concat([DTU_up,DTU_down],axis=0)

DEG_up = DEG_up.groupby('G').count()
DEG_up = DEG_up.reset_index(drop=False)
DEG_down = DEG_down.groupby('G').count()
DEG_down = DEG_down.reset_index(drop=False)

DTU_up = DTU_up.groupby('G').count()
DTU_up = DTU_up.reset_index(drop=False)
DTU_down = DTU_down.groupby('G').count()
DTU_down = DTU_down.reset_index(drop=False)



fig = plt.figure(figsize = (6,4))

gs = gridspec.GridSpec(nrows=2, ncols=5, height_ratios=[3,0.5],width_ratios=[1,2,1,2,1],wspace=0.7,hspace=0.3)


ax1=plt.subplot(gs[1,1:4])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'Number of genes', ha='center', va='center', size=15)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2=plt.subplot(gs[0,2])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'G1-G2\n\nG2-G3\n\nG3-G4', ha='center', va='center', size=20)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)


ax3=plt.subplot(gs[0,0:2])
plt.text(450, 2, 'Up', ha='center', va='center', size=20, color='red')
plt.ylim(1,0)
plt.xlim(-50,600)


ax4=plt.subplot(gs[0,3:5])
plt.text(430, 2, 'Down', ha='center', va='center', size=20, color='blue')
plt.ylim(1,0)
plt.xlim(-50,600)

sns.pointplot(data = DEG_up, x= 'gene', y= 'G',color='red', ax=ax3)
sns.pointplot(data = DTU_up, x= 'gene', y= 'G',color='pink', ax=ax3)
ax3.invert_xaxis()
ax3.axes.get_yaxis().set_visible(False)
ax3.set(xlabel=None)

sns.pointplot(data = DEG_down, x= 'gene', y= 'G',color='blue', ax=ax4)
sns.pointplot(data = DTU_down, x= 'gene', y= 'G',color='skyblue', ax=ax4)
ax4.axes.get_yaxis().set_visible(False)
ax4.set(xlabel=None)

plt.show()

## Figure S1d

In [ ]:
# pairwise DEG - 3stage_covered_genes

DEG_1 = pd.merge(DEG_G1_G2_down,DEG_G2_G3_up,how='inner',on=['gene'])
DEG_1 = pd.merge(DEG_1,DEG_G3_G4_down,how='inner',on=['gene'])

DEG_2 = pd.merge(DEG_G1_G2_up,DEG_G2_G3_down,how='inner',on=['gene'])
DEG_2 = pd.merge(DEG_2,DEG_G3_G4_up,how='inner',on=['gene'])

DEG_3 = pd.merge(DEG_G1_G2_up,DEG_G2_G3_up,how='inner',on=['gene'])
DEG_3 = pd.merge(DEG_3,DEG_G3_G4_down,how='inner',on=['gene'])



DEG_sel = pd.concat([DEG_G1_G2_up,DEG_G1_G2_down,DEG_G2_G3_up,DEG_G2_G3_down,DEG_G3_G4_up,DEG_G3_G4_down],axis=0)
DEG_sel = DEG_sel[DEG_sel['padj'] < 0.05]
DEG_sel['DEG'] = DEG_sel.apply(lambda x: 'UP' if x['log2FoldChange'] > 0 else 'DOWN', axis=1)
DEG_sel['abs_logFC'] = abs(DEG_sel['log2FoldChange'])
DEG_sel = DEG_sel[DEG_sel['abs_logFC'] > 1]

DEG_sel_1 = DEG_sel[DEG_sel['gene'].isin(DEG_1['gene'])]
DEG_sel_1['DEG_G'] = 'DEG_1'
DEG_sel_2 = DEG_sel[DEG_sel['gene'].isin(DEG_2['gene'])]
DEG_sel_2['DEG_G'] = 'DEG_2'
DEG_sel_3 = DEG_sel[DEG_sel['gene'].isin(DEG_3['gene'])]
DEG_sel_3['DEG_G'] = 'DEG_3'

DEG_sel = pd.concat([DEG_sel_1,DEG_sel_2,DEG_sel_3],axis=0)
DEG_sel['Group_N'] = DEG_sel.apply(lambda x: 1 if x['Group'] == 'G1G2' else 2  if x['Group'] == 'G2G3' else 3 , axis=1)
DEG_sel = DEG_sel[~DEG_sel['gene'].str.startswith('LOC')]

N = int(len(DEG_sel)/3)


fig = plt.figure(num=4,figsize = (4.3,10))
gs = gridspec.GridSpec(nrows=2+N, ncols=5, height_ratios=[0.3,0.3]+[1]*N,width_ratios=[0.6,2.7,2.7,2.7,4],hspace=0,wspace=0)

ax1 = plt.subplot(gs[0,1:4])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2 = plt.subplot(gs[2:23,1:4])
ax5 = plt.subplot(gs[2:23,0])

#ax1
cate = [1, 2, 3]
group = ['G1','G2','G3']
df = pd.DataFrame({'group': group, 'cate': cate })
df_heatmap = df.pivot_table(values='cate',columns='group')
sns.heatmap(df_heatmap,cmap=['#BFB788','#9FE3BB','#CECAEB'],annot=False, ax=ax1, cbar=False)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)

#ax2
sns.scatterplot(data=DEG_sel, x='Group_N', y='gene', hue='DEG',palette=['blue','red'],size='abs_logFC',sizes=(100,300), ax=ax2)
ax2.get_legend().remove()
ax2.set_xticks([])
ax2.axes.get_yaxis().set_visible(False)
ax2.set(xlim=(0.5,3.5))
ax2.set(ylim=(20.5,-0.5))
ax2.axhline(y=7.5, color='k',linewidth=1)
ax2.axhline(y=17.5, color='k',linewidth=1)
ax2.axvline(x=1.5, color='k',linewidth=1)
ax2.axvline(x=2.5, color='k',linewidth=1)
ax2.set_ylabel('',fontsize=0)
ax2.set_xlabel('',fontsize=0)

#ax3
GR = DEG_sel[['DEG_G']]
GR =GR.replace('DEG_1', 0) 
GR =GR.replace('DEG_2', 1)
GR =GR.replace('DEG_3', 2)

sns.heatmap(GR,linewidths=0,cmap = ['blue','brown','orange'],annot=False, ax=ax5, cbar=False)
ax5.axes.get_xaxis().set_visible(False)
ax5.axes.get_yaxis().set_visible(False)


n=6
g=2
for i in DEG_sel['gene'].unique():
    globals()['ax{}'.format(n)] = plt.subplot(gs[g,4])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, i, ha='center', va='center', size=10)
    n+=1
    g+=1
    
plt.show()

# Figure S2d

In [ ]:
# pairwise DTU - 2stage_covered_genes

DTU_sel = DTU_ALL

DTU_sel['DTU'] =DTU_sel.apply(lambda x: 'UP' if x['dIF'] > 0 else 'DOWN', axis=1)
DTU_sel['abs_dIF'] = abs(DTU_sel['dIF'])

DTU_1 = pd.merge(DTU_sel.groupby('G').get_group('G2_G3_down'),DTU_sel.groupby('G').get_group('G3_G4_up'),how='inner',on=['gene'])
DTU_2 = pd.merge(DTU_sel.groupby('G').get_group('G2_G3_up'),DTU_sel.groupby('G').get_group('G3_G4_down'),how='inner',on=['gene'])
DTU_3 = pd.merge(DTU_sel.groupby('G').get_group('G1_G2_down'),DTU_sel.groupby('G').get_group('G2_G3_up'),how='inner',on=['gene'])
DTU_4 = pd.merge(DTU_sel.groupby('G').get_group('G1_G2_up'),DTU_sel.groupby('G').get_group('G2_G3_down'),how='inner',on=['gene'])

DTU_sel_1 = DTU_sel[DTU_sel['gene'].isin(DTU_1['gene'])]
DTU_sel_1['DTU_G'] = 'DTU_1'
DTU_sel_2 = DTU_sel[DTU_sel['gene'].isin(DTU_2['gene'])]
DTU_sel_2['DTU_G'] = 'DTU_2'
DTU_sel_3 = DTU_sel[DTU_sel['gene'].isin(DTU_3['gene'])]
DTU_sel_3['DTU_G'] = 'DTU_3'
DTU_sel_4 = DTU_sel[DTU_sel['gene'].isin(DTU_4['gene'])]
DTU_sel_4['DTU_G'] = 'DTU_4'

DTU_sel = pd.concat([DTU_sel_1,DTU_sel_2,DTU_sel_3,DTU_sel_4],axis=0)
DTU_sel['Group_N'] = DTU_sel.apply(lambda x: 1 if x['Group'] == 'G1G2' else 2  if x['Group'] == 'G2G3' else 3 , axis=1)


N = int(len(DTU_sel)/2)

fig = plt.figure(num=4,figsize = (4,7))
gs = gridspec.GridSpec(nrows=2+N, ncols=5, height_ratios=[0.3,0.3]+[1]*N,width_ratios=[10,3,3,3,1],hspace=0,wspace=0)

ax1 = plt.subplot(gs[0,1:4])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2 = plt.subplot(gs[2:18,1:4])
ax3 = plt.subplot(gs[2:18,4])


#ax1
cate = [1, 2, 3]
group = ['G1','G2','G3']
df = pd.DataFrame({'group': group, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='group')

sns.heatmap(df_heatmap,cmap=['#BFB788','#9FE3BB','#CECAEB'],annot=False, ax=ax1, cbar=False)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)


#ax2
sns.scatterplot(data=DTU_sel, x='Group_N', y='gene', hue='DTU',palette=['skyblue','pink'],size='abs_dIF',sizes=(20,200), ax=ax2)
ax2.get_legend().remove()
ax2.set_xticks([])
ax2.axes.get_yaxis().set_visible(False)
ax2.axes.get_xaxis().set_visible(False)
ax2.set(xlim=(0.5,3.5))
ax2.axhline(y=3.375, color='k',linewidth=1)
ax2.axhline(y=6.47, color='k',linewidth=1)
ax2.axhline(y=10.59, color='k',linewidth=1)
ax2.axvline(x=1.5, color='k',linewidth=1)
ax2.axvline(x=2.5, color='k',linewidth=1)

#ax3
GR = DTU_sel[['DTU_G']]
GR =GR.replace('DTU_1', 0) 
GR =GR.replace('DTU_2', 1)
GR =GR.replace('DTU_3', 2)
GR =GR.replace('DTU_4', 3)

sns.heatmap(GR,linewidths=0,cmap = ['orange','brown','blue','green'],annot=False, ax=ax3, cbar=False)
ax3.axes.get_xaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)

n=6
g=2
for i in DTU_sel['gene'].unique():
    globals()['ax{}'.format(n)] = plt.subplot(gs[g,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, i, ha='center', va='center', size=8)
    n+=1
    g+=1

    
plt.show()

## Figure 1b

In [ ]:
DEG_C_ALL = DEG_A_ALL
DEG_C_ALL['n_index'] = DEG_C_ALL['gene'] + '_' + DEG_C_ALL['G']
DEG_C_ALL['n_index'] = DEG_C_ALL['n_index'].astype(str)
DEG_C_ALL = DEG_C_ALL.set_index('gene')
DEG_C_ALL['-log10_padj'] = abs(np.log10(DEG_C_ALL['padj']))
DEG_C_ALL = DEG_C_ALL .sort_values(by=['G','-log10_padj'], ascending=[True,False])


fig = plt.figure(num=4,figsize = (5,15))
gs = gridspec.GridSpec(nrows=4, ncols=3, height_ratios=[1,0,80,2],width_ratios=[4,1,3],hspace=0.02)

ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[2,0])
plt.yticks(fontsize=10)
ax3 = plt.subplot(gs[2,1])

ax4 = plt.subplot(gs[2,2])
plt.xticks(fontsize=10)


#ax1
cate = [1, 2, 3, 4,5]
name = ['aging','oncogene','TSG','fusion','effect']
df = pd.DataFrame({'name': name, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='name')
sns.heatmap(df_heatmap,cmap=['teal','chocolate','maroon','tan','b'],annot=False, ax=ax1, cbar=False)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)


#ax2
CA = DEG_C_ALL[['aging','oncogene','TSG','fusion','effect']]
CA =CA.replace('N', 0) 
CA =CA.replace('Y', 1)
CA =CA.replace('H', 2) #inhibits (senescence effect)
CA =CA.replace('D', 3) #induces (senescence effect)

sns.heatmap(CA,linewidths=3,cmap=['lightgray','grey','pink','lightblue'],annot=False, ax=ax2, cbar=False)
ax2.axes.get_xaxis().set_visible(False)
ax2.axhline(y=0, color='k',linewidth=1)
ax2.axhline(y=72.9, color='k',linewidth=1)
ax2.axvline(x=0, color='k',linewidth=1)
ax2.axvline(x=4.9, color='k',linewidth=1)


#ax3
GR = DEG_C_ALL[['G']]
GR =GR.replace('0_G1_G2_up', 0) 
GR =GR.replace('1_G1_G2_down', 0)
GR =GR.replace('2_G2_G3_up', 1)
GR =GR.replace('3_G2_G3_down', 1)
GR =GR.replace('4_G3_G4_up', 2)
GR =GR.replace('5_G3_G4_down', 2)
sns.heatmap(GR,linewidths=0,cmap = ['#BFB788','#9FE3BB','#CECAEB'],annot=False, ax=ax3, cbar=False)
ax3.axes.get_xaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)


#ax4
DEG_C_ALL['log2FC'] = 'A'
DEG_C_ALL['DEG'] = DEG_C_ALL.apply(lambda x: 'UP' if x['log2FoldChange'] > 0 else 'DOWN', axis=1)
DEG_C_ALL['abs_logFC'] = abs(DEG_C_ALL['log2FoldChange'])

sns.scatterplot(data=DEG_C_ALL, x='-log10_padj', y='n_index', hue='DEG', palette=['red','blue'], size='abs_logFC',sizes=(20,100), ax=ax4)
ax4.set(ylim=(72.5,-0.5))
ax4.axes.get_yaxis().set_visible(False)
ax4.set_xlabel('-log10_padj',fontsize=15)
ax4.get_legend().remove()


plt.show()

## Figure S2c

In [ ]:

ALL = pd.DataFrame()
for i in ['G1G2','G2G3','G3G4']:
    DEG = DEG_ALL[DEG_ALL['Group'] == i]
    DTU = DTU_ALL[DTU_ALL['Group'] == i]
    DTU['gene'] = DTU['gene'].str.split('_').str[0]
    for x in DEG['G'].unique():
        for y in DTU['G'].unique():
            
            G = x.split('_')[2]
            T = y.split('_')[2]
            globals()['{}_{}_{}'.format(i,G,T)] = pd.merge(DEG.groupby('G').get_group(x),DTU.groupby('G').get_group(y),how='inner',on=['gene'])
            
            ALL = pd.concat([ALL,globals()['{}_{}_{}'.format(i,G,T)]],axis=0)
            
    #globals()['{}_DEG_up'.format(i)] = DEG[DEG['gene'].isin(G1G2_up_up['gene'])]
    #globals()['{}_DTU_up'.format(i)] = DTU[DTU['gene'].isin(G1G2_up_up['gene'])]
    
    #globals()['{}_DEG_down'.format(i)] = DEG[DEG['gene'].isin(G1G2_down_down['gene'])]
    #globals()['{}_DTU_down'.format(i)] = DTU[DTU['gene'].isin(G1G2_down_down['gene'])]


ALL = ALL.reset_index(drop=True)

ALL['-log10_padj'] = abs(np.log10(ALL['padj']))
ALL['-log10_qvalue'] = abs(np.log10(ALL['isoform_switch_q_value']))

ALL = ALL.sort_values(by=['condition_1','condition_2','gene'], ascending=[True,True,True])


ALL['log2FC'] = 'A'
ALL['DEG'] = ALL.apply(lambda x: 'UP' if x['log2FoldChange'] > 2 else 'DOWN', axis=1)
ALL['abs_logFC'] = abs(ALL['log2FoldChange'])
ALL = ALL[ALL['padj']<0.01]
ALL = ALL[ALL['abs_logFC']>1]

ALL['dIF_A'] = 'A'
ALL['DTU'] = ALL.apply(lambda x: 'UP' if x['dIF'] > 0.05 else 'DOWN', axis=1)
ALL['abs_dIF'] = abs(ALL['dIF'])
ALL = ALL[ALL['isoform_switch_q_value']<0.01]
ALL = ALL[ALL['abs_dIF']>0.05]

ALL = ALL.round(2)

fig = plt.figure(figsize = (12,6.5))

gs = gridspec.GridSpec(nrows=11, ncols=5, height_ratios=[2,0,1,1,1,1,1,1,1,1,0],width_ratios=[2,1,1,1,2],hspace=0)


ax1 = plt.subplot(gs[0,0:1])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'DEGs', ha='center', va='center', size=30)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax3 = plt.subplot(gs[0,4:6])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'DTUs', ha='center', va='center', size=30)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax4 = plt.subplot(gs[2:4,2])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'G1\n-\nG2', ha='center', va='center', size=20)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_facecolor('#BFB788') #배경색

ax5 = plt.subplot(gs[4:8,2])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'G2\n-\nG3', ha='center', va='center', size=20)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_facecolor('#9FE3BB') #배경색

ax6 = plt.subplot(gs[8:10,2])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'G3\n-\nG4', ha='center', va='center', size=20)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_facecolor('#CECAEB') #배경색


ax8 = plt.subplot(gs[1:10,0])
plt.xticks(fontsize=10)
ax9 = plt.subplot(gs[1:10,4])
plt.xticks(fontsize=10)


n=11
g=2
for i in ALL['gene']:
    globals()['ax{}'.format(n)] = plt.subplot(gs[g,1])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, i, ha='center', va='center', size=20)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    n+=1
    g+=1
    
    
n=25
g=2
for j in ALL['isoform_id']:
    j = j.split('ENSMFAT000000')[1]
    j = 'E::'+j
    globals()['ax{}'.format(n)] = plt.subplot(gs[g,3])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, j, ha='center', va='center', size=20)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    n+=1
    g+=1


##ax8
sns.scatterplot(data=ALL, x='-log10_padj' ,y=ALL.index.astype(str), hue='DEG',palette=['blue','red'], size='abs_logFC',sizes=(20,200), ax=ax8)
sns.despine(top=False,left=False,bottom=False,right=False,ax=ax8)
ax8.set(xlim=(0,25))
ax8.invert_xaxis()
ax8.get_legend().remove()
ax8.set_xlabel('-log10_padj',fontsize=15)
ax8.axes.get_yaxis().set_visible(False)


##ax9
sns.scatterplot(data=ALL, x='-log10_qvalue' ,y=ALL.index.astype(str), hue='DTU',palette=['pink','skyblue'], size='abs_dIF',sizes=(20,200), ax=ax9)
sns.despine(top=False,left=False,bottom=False,right=False,ax=ax9)
ax9.set(xlim=(0,6))
ax9.get_legend().remove()
ax9.set_xlabel('-log10_qvalue',fontsize=15)
ax9.axes.get_yaxis().set_visible(False) 



# background axes object for plotting the vertical line
ax =  fig.add_subplot(gs[:, :])
# set background color to transparent and turn off the frame
ax.patch.set_alpha(0)
ax.axis("off")

plt.show()

## Figure S2e & S2g

In [ ]:
DTU_GG = pd.concat([DTU_G1_G2, DTU_G2_G3, DTU_G3_G4],axis=0)

CPM = GeneCPM
CPM = file_parsing(CPM)


def Geneplot(gene,C1,C2,col1,col2):
    
    sel_CPM = CPM[CPM['gene'] == gene]
    sel_CPM = sel_CPM[(sel_CPM['Group'] == C1) | (sel_CPM['Group'] == C2)]
    
    sel_DTU = DTU_GG[(DTU_GG['gene'] == gene) & (DTU_GG['condition_1'] == C1) & (DTU_GG['condition_2'] == C2)] 
    DTU = sel_DTU[sel_DTU['isoform_switch_q_value'] < 0.05]
    DTU['isoform'] = DTU.isoform_id.str.replace('ENSMFAT000000','E::',case=False)
    DTU = DTU[['isoform','isoform_switch_q_value','IF1','IF2']]
    DTU = DTU.rename(columns={'IF1':C1,'IF2':C2,'isoform_switch_q_value':'qvalue'})
    DTU_melt = pd.melt(DTU, id_vars=['isoform','qvalue'])
    DTU_melt['qvalue'] = DTU_melt['qvalue'].astype(float)
    
  
    fig = plt.figure(figsize = (5,3))

    gs = gridspec.GridSpec(nrows=2, ncols=3, height_ratios=[0.5,3],width_ratios=[1,1,1],hspace=0,wspace=0.7)
         
    ax0 = plt.subplot(gs[1,0])
    plt.yticks(fontsize=10)
    sns.swarmplot(data=sel_CPM, x='Group', y='value', color='darkgrey', order=[C1,C2],size=5, ax=ax0)
    sns.boxplot(data=sel_CPM, x='Group', y='value', showmeans=True, meanline=True, meanprops={'color': 'k', 'ls': '-', 'lw': 2},medianprops={'visible': False}, whiskerprops={'visible': False},zorder=10,showfliers=False,showbox=False,showcaps=False, order=[C1,C2],ax=ax0)
    ax0.set_ylabel('CPM',fontsize=10)
    ax0.set_xticklabels([C1,C2],fontsize=12)
    ax0.set(xlabel=None)

    DTU_1 = DTU_melt[DTU_melt['isoform'] == DTU_melt['isoform'].unique()[0]]
    DTU_2 = DTU_melt[DTU_melt['isoform'] == DTU_melt['isoform'].unique()[1]]

    ax1 = plt.subplot(gs[1,1])
    plt.ylim(0,1.0)
    sns.barplot(data=DTU_1, x='variable', y='value',palette=[col1,col2], ax=ax1)
    ax1.set_ylabel('isoform fraction',fontsize=10)
    ax1.set_xticklabels([C1,C2],fontsize=12)
    ax1.set_yticklabels(['0','0.2','0.4','0.6','0.8'],fontsize=10)
    ax1.set(xlabel=None)

    ax2 = plt.subplot(gs[1,2])
    plt.ylim(0,1.0)
    sns.barplot(data=DTU_2, x='variable', y='value',palette=[col1,col2], ax=ax2)
    ax2.set_ylabel('isoform fraction',fontsize=10)
    ax2.set_xticklabels([C1,C2],fontsize=12)
    ax2.set_yticklabels(['0','0.2','0.4','0.6','0.8'],fontsize=10)
    ax2.set(xlabel=None)
    
    ax3= plt.subplot(gs[0,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, 'CPM', ha='center', va='center', size=10, alpha=1)   
    
    ax4 = plt.subplot(gs[0,1])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, DTU_1.iloc[0,0], ha='center', va='center', size=10, alpha=1)
    
    ax5 = plt.subplot(gs[0,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, DTU_2.iloc[0,0], ha='center', va='center', size=10, alpha=1)   
    
    
    plt.subplots_adjust(wspace=0,hspace=0)

    plt.show()

Geneplot('CTSW','G1','G2','#ED8080','#90EE90')
Geneplot('CTSW','G2','G3','#90EE90','#ADD8E6') 
Geneplot('OAS1','G3','G4','#ADD8E6','#EFBBF0') 

## Figure 2b

In [ ]:
file = './source_data/WGCNA/DEmRNA_age_greenyellow_WGCNA.txt'
greenyellow = pd.read_csv(file, sep = '\t', header = None,names=['gene_id'])

file = './source_data/WGCNA/DEmRNA_age_turquoise_WGCNA.txt'
turquoise= pd.read_csv(file, sep = '\t', header = None,names=['gene_id'])

WGCNA = pd.concat([greenyellow,turquoise], axis=0)
WGCNA['gene'] = WGCNA.apply(lambda x: x.gene_id.split('_')[0], axis=1)

for num, i in zip([1,2,3,4,5,6,7],[DEG_G1,DEG_G2,DEG_G3,DEG_G4,DEG_G5,DEG_G6,DEG_G7]):
    i['group'] = num
    
ELTC_ALL = pd.concat([DEG_G2,DEG_G3,DEG_G4,DEG_G6],axis=0)

In [ ]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(5,5))
Course1=ELTC_ALL['gene']
Course2=WGCNA['gene']

vd = venn2([set(Course1), set(Course2)],set_labels=('degpatterns', 'WGCNA'),
           set_colors=('#F4ACB7','#c4e6ff'),
           alpha=0.8
          )
venn2_circles([set(Course1), set(Course2)], linestyle='-', linewidth=1, color='grey')
for text in vd.set_labels:  #change label size
    text.set_fontsize(15);
for text in vd.subset_labels:  #change number size
    text.set_fontsize(15)

plt.show()

## Figure 2d

In [ ]:
ELTC_WGCNA = pd.merge(ELTC_ALL,WGCNA,how='inner',on=['gene'],indicator=True)
DEG_norm_count_mfa_P = file_parsing(DEG_norm_count_mfa)


def zs(df):
    DF = pd.DataFrame()
    for i in df['gene'].unique():
        G = df[df['gene'] == i]       
        G = G.copy()
        G['z'] = stats.zscore(G['value'])
        DF = pd.concat([DF,G], axis=0)
    return DF

EW_norm = pd.merge(DEG_norm_count_mfa_P,ELTC_WGCNA,how='inner',on=['gene'],indicator=False) 

se = EW_norm.groupby(['gene','age','group'])['value'].mean()
se = se.reset_index(drop=False)
se = zs(se)
se_pivot = se.pivot_table(index=['gene','group'],values='z',columns='age')
SEL = se_pivot
SEL = SEL.reset_index(drop=False)
SEL = SEL[(SEL['group'] == 2) | (SEL['group'] == 3) | (SEL['group'] == 6)]
SEL = SEL.sort_values(by=['group'],ascending=[True])


fig = plt.figure(figsize = (15,15))

gs = gridspec.GridSpec(nrows=2, ncols=3, height_ratios=[1,40],width_ratios=[48,1,2],hspace=0.05,wspace=0.05)
ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[1,0])
ax3 = plt.subplot(gs[1,1])


#ax1
cate = [1, 2, 3, 4]
group = ['G1','G2','G3','G4']
df = pd.DataFrame({'group': group, 'cate': cate })
df_heatmap = df.pivot_table(values='cate',columns='group')

sns.heatmap(df_heatmap,cmap=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'],annot=False, cbar=False, ax=ax1)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)

#ax2
SEL_value = SEL.iloc[:,2:]
sns.heatmap(SEL_value, linewidths=0, cmap = 'YlGnBu', annot=False, cbar=False, ax=ax2)
ax2.axes.get_xaxis().set_visible(False)
ax2.axes.get_yaxis().set_visible(False)

#ax3
SEL_Group = SEL['group']
SEL_Group = SEL_Group.reset_index(drop=False)
SEL_Group = SEL_Group[['group']]
SEL_Group['ord'] = [1 if SEL_Group.iloc[idx]['group'] == 2 
                    else 2  if SEL_Group.iloc[idx]['group'] == 3 
                    else 3 for idx in SEL_Group.index]
SEL_Group = SEL_Group[['ord']]
sns.heatmap(SEL_Group, linewidths=0,cmap = ['#66c2a5','#8da0cb','#deb887'],annot=False, cbar=False, ax=ax3)
ax3.axes.get_xaxis().set_visible(False)
ax3.axes.get_yaxis().set_visible(False)

plt.show()

## Fig. 3

In [ ]:
TIMER_group['immune'] = TIMER_group.cell_type.str.split('_').str[0]
TIMER_group['program'] = TIMER_group.cell_type.str.split('_').str[1]
T = TIMER_group[TIMER_group['program']=='XCELL']

## Figure 3d-e

In [ ]:
def TIMER_Graph(T_df,T_name,C_df,C_name):

    fig = plt.figure(figsize = (25,10))

    gs = gridspec.GridSpec(4,1) 
    
    gs0 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=8, height_ratios=[0.5,2],width_ratios=[0.2,1,1,1,1,1,1,1],hspace=0,wspace=0.35, subplot_spec=gs[0])
    gs1 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=8, height_ratios=[0.5,2],width_ratios=[0.2,1,1,1,1,1,1,1],hspace=0,wspace=0.35, subplot_spec=gs[1])


    plt.subplot(gs0[0:2,0])
    plt.xticks([])
    plt.yticks([])
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.text(0.5, 0.5, 'TIMER\ninfiltration level', ha='center', va='center', size=15,  rotation=90)
    
    plt.subplot(gs1[0:2,0])
    plt.xticks([])
    plt.yticks([])
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.text(0.5, 0.5, 'CIBERSORT_ABS\ninfiltration level', ha='center', va='center', size=15,  rotation=90)
    
    
    a = 1
    for im in T_df['immune'].unique():
        
        dt = T_df[T_df['immune'] == im]
        
        
        plt.subplot(gs0[0,a])
        plt.xticks([])
        plt.yticks([])
        plt.text(0.5, 0.5, im, ha='center', va='center', size=10)
        plt.gca().set_facecolor('lightgrey') #배경색
        
        plt.subplot(gs0[1,a])
        ax1 = sns.barplot(data = dt, x='Group', y='value', palette=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'])

        plt.yticks(fontsize=10)
        plt.xticks(fontsize=15)
        plt.xlabel('')
        plt.ylabel('')
        
        for i, tick in enumerate(ax1.yaxis.get_ticklabels()):
            if i % 2 != 0:
                tick.set_visible(False) 

        a += 1
    

    b = 1
    for im in C_df['immune'].unique():
        
        
        dt = C_df[C_df['immune'] == im]
        
        plt.subplot(gs1[0,b])
        plt.xticks([])
        plt.yticks([])
        plt.text(0.5, 0.5, im, ha='center', va='center', size=10)
        plt.gca().set_facecolor('lightgrey') #배경색
        
        plt.subplot(gs1[1,b])
        ax2 = sns.barplot(data = dt, x='Group', y='value', palette=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'])

        plt.yticks(fontsize=10)
        plt.xticks(fontsize=15)
        plt.xlabel('')
        plt.ylabel('')
        for i, tick in enumerate(ax2.yaxis.get_ticklabels()):
            if i % 2 != 0:
                tick.set_visible(False)

        b += 1

    plt.show()


TG = TIMER_group[['immune','program','G1','G2','G3','G4']]
T = TG[TG['program']=='TIMER']
T = pd.melt(T, id_vars=['immune','program'])
T = T.rename(columns={'variable':'Group'})

C = TG[TG['program']=='CIBERSORT-ABS']
C = pd.melt(C, id_vars=['immune','program'])
C = C.rename(columns={'variable':'Group'})


D = C.pivot(index='immune', columns='Group', values='value')
D = D[~((D['G1'] == 0) & (D['G2'] == 0) & (D['G3'] == 0) & (D['G4'] == 0))]
D = D.reset_index(drop=False)

D = D[(D.immune.str.contains('B cell')) | (D.immune.str.contains('T cell') & ~(D.immune.str.contains('CD8'))) | (D.immune.str.contains('NK cell'))]

E = pd.DataFrame()
for i in D['immune'].unique():
    
    e = C[C['immune'] == i]
    E = pd.concat([E,e],axis=0)
    
TIMER_Graph(T,'TIMER',E,'CIBERSORT_ABS')

## Figure S5a

In [ ]:
def TIMER_Graph(T_df,T_name,C_df,C_name):

    fig = plt.figure(figsize = (21,7))

    gs = gridspec.GridSpec(4,1) 
    
    gs0 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=7, height_ratios=[0.5,2],width_ratios=[1,1,1,1,1,1,1],hspace=0, subplot_spec=gs[0])
    gs1 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=7, height_ratios=[0.5,2],width_ratios=[1,1,1,1,1,1,1],hspace=0, subplot_spec=gs[1])
    gs2 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=7, height_ratios=[0.5,2],width_ratios=[1,1,1,1,1,1,1],hspace=0, subplot_spec=gs[2])
    gs3 = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=7, height_ratios=[0.5,2],width_ratios=[1,1,1,1,1,1,1],hspace=0, subplot_spec=gs[3])

    
    a = 1
    for im in T_df['immune'].unique():        
        dt = T_df[T_df['immune'] == im]
        a += 1
    

    b = 1
    c = 1
#    for im in C_df['immune'].unique():
    for im in ['Eosinophil','Macrophage M0','Mast cell resting','Monocyte','Myeloid dendritic cell resting',
               'NK cell resting','Neutrophil','T cell CD8+']:

        
        if 6 > b :
            gsN = gs1
        elif b == 6:
            c = 1
            gsN = gs2
        elif 13 > b :
            gsN = gs2
        elif b == 13:
            c = 1
            gsN = gs3
        else:
            gsN = gs3
            
        
        dt = C_df[C_df['immune'] == im]
        
        if dt.empty:
            continue
        else:
            pass
        
        plt.subplot(gsN[0,c])
        plt.xticks([])
        plt.yticks([])
        plt.text(0.5, 0.5, im, ha='center', va='center', size=12)
        plt.gca().set_facecolor('lightgrey') #배경색
        
        globals()['ax{}'.format(c)] = plt.subplot(gsN[1,c])
        sns.barplot(data = dt, x='Group', y='value' ,palette=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'])
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=8)
        plt.xlabel('')
        plt.ylabel('')
        for i, tick in enumerate(globals()['ax{}'.format(c)].yaxis.get_ticklabels()):
            if i % 2 != 0:
                tick.set_visible(False)  

        c += 1
        b += 1
    
    plt.show()


TG = TIMER_group[['immune','program','G1','G2','G3','G4']]
T = TG[TG['program']=='TIMER']
T = pd.melt(T, id_vars=['immune','program'])
T = T.rename(columns={'variable':'Group'})

C = TG[TG['program']=='CIBERSORT-ABS']
C = pd.melt(C, id_vars=['immune','program'])
C = C.rename(columns={'variable':'Group'})


D = C.pivot(index='immune', columns='Group', values='value')
D = D[~((D['G1'] == 0) & (D['G2'] == 0) & (D['G3'] == 0) & (D['G4'] == 0))]
D = D.reset_index(drop=False)

E = pd.DataFrame()
for i in D['immune'].unique():
    
    e = C[C['immune'] == i]
    E = pd.concat([E,e],axis=0)

TIMER_Graph(T,'TIMER',E,'CIBERSORT_ABS')

In [ ]:
DEG_norm_count_mfa['gene'] = DEG_norm_count_mfa.apply(lambda x: x.name.split('_')[0], axis=1)
DICT = {'G1':DEG_G1,'G2':DEG_G2,'G3':DEG_G3,'G4':DEG_G4,'G5':DEG_G5,'G6':DEG_G6,'G7':DEG_G7}

ALL = pd.DataFrame()

for key in DICT:
    
    DF = DICT[key]
    globals()["{}_annot".format(key)] =  pd.merge(DF,annot_gene,how='inner',on=['gene'])
    DF_annot = globals()["{}_annot".format(key)]
    
    globals()["{}_annot_ncount".format(key)] =  pd.merge(DF_annot,DEG_norm_count_mfa,how='inner',on=['gene'])
    DF_annot_ncount = globals()["{}_annot_ncount".format(key)]
    DF_annot_ncount['Group'] = key
    
    ALL = pd.concat([ALL,DF_annot_ncount],axis=0)
    
ALL = ALL.set_index('gene')
ALL['-log10_padj'] = abs(np.log10(ALL['padj']))
ALL = ALL.sort_values(by=['Group','-log10_padj'], ascending=[True,False])

CA = ALL[['aging','oncogene','TSG','fusion','effect']]
CA =CA.replace('N', 0) 
CA =CA.replace('Y', 1)
CA =CA.replace('H', 2) #inhibits (senescence effect)
CA =CA.replace('D', 3) #induces (senescence effect)

hub_605 = hubgenes_605
hub_605 = hub_605.rename(columns={'Name':'gene'})
hub_605 = pd.merge(hub_605,CA,how='left', on='gene')
hub_605['gene'].unique()
hub_605_sel = hub_605[(hub_605['oncogene'] == 1) | (hub_605['TSG'] == 1)]
hub_605_sel = hub_605_sel[['gene','Score','Group','Rank','oncogene','TSG']]

## Figure S7b

In [ ]:
CB=hub_605_sel
fig = plt.figure(figsize = (2.5,10))

gs = gridspec.GridSpec(nrows=43, ncols=3, height_ratios=[0.5,0.5,
                                                         1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                                                         1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                                                         1],
                       width_ratios=[3,4,1],hspace=0,wspace=0)


n=0
a=2
for g,G,r in zip(CB['gene'].tolist(),CB['Group'].tolist(),CB['Rank'].tolist()):
                           
    globals()['ax{}'.format(n)] = plt.subplot(gs[a,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, g, ha='center', va='center', size=10)

    if G == 'human_WGCNA_group2':
        plt.gca().set_facecolor('#66c2a5') #배경색
    elif G == 'human_WGCNA_group3':
        plt.gca().set_facecolor('#8da0cb') #배경
    else:
        plt.gca().set_facecolor('#deb887') #배경색
        
    globals()['ax{}'.format(n)] = plt.subplot(gs[a,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, r, ha='center', va='center', size=10)
               
    n+=1
    a+=1

ax20 = plt.subplot(gs[0,1])
ax21 = plt.subplot(gs[2:43,1])

cate = [1, 2]
name = ['oncogene','TSG']
df = pd.DataFrame({'name': name, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='name')
sns.heatmap(df_heatmap,cmap=['slateblue','chocolate','darkkhaki','tan','b'],annot=False, ax=ax20, cbar=False)
ax20.axes.get_xaxis().set_visible(False)
ax20.axes.get_yaxis().set_visible(False)


#ax3

S = CB[['oncogene','TSG']]

sns.heatmap(S,linewidths=3,cmap=['lightgray','grey'],annot=False, ax=ax21, cbar=False)
ax21.axes.get_xaxis().set_visible(False)
ax21.axes.get_yaxis().set_visible(False)
ax21.axhline(y=0, color='k',linewidth=3)
ax21.axhline(y=6, color='k',linewidth=1.5)
ax21.axhline(y=37, color='k',linewidth=1.5)
ax21.axhline(y=41, color='k',linewidth=1.5)
ax21.axvline(x=0, color='k',linewidth=3)
ax21.axvline(x=2, color='k',linewidth=3)

plt.show()

## Figure 4c

In [ ]:
ALL_sel = ALL[ALL['Group'].isin(['G2','G3','G4','G6'])]
ALL_sel.to_csv('./source_data/annotation_files/DEG_annot.txt',sep='\t',index=False,header=True)

fig = plt.figure(figsize = (10,25))

gs = gridspec.GridSpec(nrows=2, ncols=4, height_ratios=[1,142],width_ratios=[5,12,0.5,3])
ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[0,1]) 
ax3 = plt.subplot(gs[1,0])
plt.yticks(fontsize=8)
ax4 = plt.subplot(gs[1,1])
ax5 = plt.subplot(gs[1,2])
ax6 = plt.subplot(gs[1,3])
plt.xticks(fontsize=10)


#ax1

cate = [1, 2, 3, 4, 5]
name = ['aging','oncogene','TSG','fusion','effect']
df = pd.DataFrame({'name': name, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='name')
sns.heatmap(df_heatmap,cmap=['slateblue','chocolate','darkkhaki','tan','b'],annot=False, ax=ax1, cbar=False)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)


#ax2

cate = [1, 2, 3, 4]
group = ['G1','G2','G3','G4']
df = pd.DataFrame({'group': group, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='group')

sns.heatmap(df_heatmap,cmap=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'],annot=False, ax=ax2, cbar=False)
ax2.axes.get_xaxis().set_visible(False)
ax2.axes.get_yaxis().set_visible(False)


#ax3
CA = ALL_sel[['aging','oncogene','TSG','fusion','effect']]
CA =CA.replace('N', 0) 
CA =CA.replace('Y', 1)
CA =CA.replace('H', 2) #inhibits (senescence effect)
CA =CA.replace('D', 3) #induces (senescence effect)
sns.heatmap(CA,linewidths=3,cmap=['lightgray','grey','pink','lightblue'],annot=False, ax=ax3, cbar=False)
ax3.axes.get_xaxis().set_visible(False)
ax3.axhline(y=0, color='k',linewidth=3)
ax3.axhline(y=141, color='k',linewidth=3)
ax3.axvline(x=0, color='k',linewidth=3)
ax3.axvline(x=5, color='k',linewidth=3)


#ax4

def zs(df):
    DF = pd.DataFrame()
    for i in df['gene'].unique():
        G = df[df['gene'] == i]       
        G = G.copy()
        G['z'] = stats.zscore(G['value'])
        DF = pd.concat([DF,G], axis=0)
    return DF   

AGE = ALL_sel.iloc[:,8:32]
age = file_parsing(AGE)
age = age.groupby(['gene','age'])['value'].mean()
age = age.reset_index(drop=False)
age = zs(age)
age_pivot = age.pivot_table(index='gene',values='z',columns='age')
age_pivot = pd.merge(age_pivot,ALL[['Group','-log10_padj']],how='inner', on='gene')
age_pivot = age_pivot.sort_values(by=['Group','-log10_padj'], ascending=[True,False])

age_pivot = age_pivot.iloc[:,:-2]
sns.heatmap(age_pivot,linewidths=3,annot=False, ax=ax4, cbar=False,cmap='YlGnBu')
ax4.axes.get_xaxis().set_visible(False)
ax4.axes.get_yaxis().set_visible(False)
ax4.axhline(y=0, color='k',linewidth=3)
ax4.axhline(y=141, color='k',linewidth=3)
ax4.axvline(x=0, color='k',linewidth=3)
ax4.axvline(x=12, color='k',linewidth=3)

#ax5
GR = ALL_sel[['Group']]
GR =GR.replace('G1', 0) 
GR =GR.replace('G2', 1) 
GR =GR.replace('G3', 2)
GR =GR.replace('G4', 3)
GR =GR.replace('G5', 4)
GR =GR.replace('G6', 5)
GR =GR.replace('G7', 6)

sns.heatmap(GR,linewidths=0,annot=False, ax=ax5, cbar=False,cmap=['#66c2a5','#8da0cb','#a5d854','thistle','slategray','burlywood'])
ax5.axes.get_xaxis().set_visible(False)
ax5.axes.get_yaxis().set_visible(False)


#ax6
pv = ALL_sel[['padj']]
pv['-log10_padj'] = abs(np.log10(pv['padj']))
pv = pv.reset_index(drop=False)

sns.barplot(data = pv, x='-log10_padj', y='gene', color='skyblue', ax=ax6)
ax6.axes.get_yaxis().set_visible(False)

plt.show()

## Figure 4a

In [ ]:
hub = hubgenes
hub = pd.merge(hub,CA,how='left', on='gene')
hub = hub.replace(np.nan, 0)
hub['annot'] = hub['aging']+hub['oncogene']+hub['TSG']+hub['fusion']+hub['effect']
hub = hub.iloc[:,[0,1,2,3,9]]

fig = plt.figure(figsize = (3,15))
gs = gridspec.GridSpec(nrows=61, ncols=5, height_ratios=[0.5,0.3]+[1.5]*59,width_ratios=[4,1,1,5,4],hspace=0,wspace=0)

ax1 = plt.subplot(gs[2:61,1])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2 = plt.subplot(gs[2:61,3])


ax5 = plt.subplot(gs[0,2])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'R', ha='center', va='center', size=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
ax6 = plt.subplot(gs[0,3])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'Score', ha='center', va='center', size=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
ax7 = plt.subplot(gs[0,4])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0.5, 'Gene', ha='center', va='center', size=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)


#ax1
hub_G = hub[['Group']]
hub_G =hub_G.replace('human_WGCNA_group2', 0) 
hub_G =hub_G.replace('human_WGCNA_group3', 1)
hub_G =hub_G.replace('human_WGCNA_group6', 2)

sns.heatmap(hub_G,cmap=['#66c2a5','#8da0cb','#deb887'],annot=False, ax=ax1, cbar=False)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)


#ax2
hub_S = hub[['Score','gene']]

sns.scatterplot(data=hub_S, x='Score', y='gene', palette='plasma',hue='Score',size='Score',sizes=(10,100), ax=ax2)
#ax2.get_legend().remove()
ax2.legend(bbox_to_anchor=(-0.5,0.08),frameon=False)
ax2.axes.get_yaxis().set_visible(False)
ax2.axes.get_xaxis().set_visible(False)
ax2.set(xlim=(-0.18,1.2))
ax2.set(ylim=(58.5,-0.5))

n=10
a=2
for r,g,annot in zip(hub['Rank'].tolist(),hub['gene'].tolist(),hub['annot'].tolist()):
    
    globals()['ax{}'.format(n)] = plt.subplot(gs[a,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, r, ha='center', va='center', size=10)
                          
    globals()['ax{}'.format(n+52)] = plt.subplot(gs[a,4])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, g, ha='center', va='center', size=10)

    if annot > 0 and  22 > a:
        plt.gca().set_facecolor('#66c2a5')
    elif annot > 0 and  42 > a:
        plt.gca().set_facecolor('#8da0cb')
    elif annot > 0 and  63 > a:
        plt.gca().set_facecolor('#deb887')
               
    n+=1
    a+=1

#plt.rcParams["figure.autolayout"] = True
#plt.tight_layout()
plt.savefig("/home/chm/RNA/COMPLETE_PAPER/figures/figure4a.png",dpi=1000)
plt.show()

## Figure 4b

In [ ]:
import scikit_posthocs as sp
import matplotlib.gridspec as gridspec

hub_sel = hub[hub['annot']>0]

CB = pd.merge(hub_sel,CA,how='inner', on='gene')
CB = CB.iloc[:,[0,2,5,6,7,8,9]]
CB = CB.drop_duplicates(['gene'],keep='first')


fig = plt.figure(figsize = (4,6))

gs = gridspec.GridSpec(nrows=25, ncols=2, height_ratios=[0.5,0.5]+[1]*23,width_ratios=[2,4],hspace=0,wspace=0)


n=0
a=2
for g,G in zip(CB['gene'].tolist(),CB['Group'].tolist()):
                           
    globals()['ax{}'.format(n)] = plt.subplot(gs[a,0])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, g, ha='center', va='center', size=10)
    #plt.gca().spines['right'].set_visible(False)
    #plt.gca().spines['left'].set_visible(False)
    #plt.gca().spines['top'].set_visible(False)
    #plt.gca().spines['bottom'].set_visible(False)
    if G == 'human_WGCNA_group2':
        plt.gca().set_facecolor('#66c2a5') #배경색
    elif G == 'human_WGCNA_group3':
        plt.gca().set_facecolor('#8da0cb') #배경
    else:
        plt.gca().set_facecolor('#deb887') #배경색
               
    n+=1
    a+=1

ax20 = plt.subplot(gs[0,1])
ax21 = plt.subplot(gs[2:26,1])

#ax20

cate = [1, 2, 3, 4, 5]
name = ['aging','oncogene','TSG','fusion','effect']
df = pd.DataFrame({'name': name, 'cate': cate })

df_heatmap = df.pivot_table(values='cate',columns='name')
sns.heatmap(df_heatmap,cmap=['slateblue','chocolate','darkkhaki','tan','b'],annot=False, ax=ax20, cbar=False)
ax20.axes.get_xaxis().set_visible(False)
ax20.axes.get_yaxis().set_visible(False)


#ax3

S = CB[['aging','oncogene','TSG','fusion','effect']]

sns.heatmap(S,linewidths=3,cmap=['lightgray','grey','pink','lightblue'],annot=False, ax=ax21, cbar=False)
ax21.axes.get_xaxis().set_visible(False)
ax21.axes.get_yaxis().set_visible(False)
ax21.axhline(y=0, color='k',linewidth=3)
ax21.axhline(y=3, color='k',linewidth=1.5)
ax21.axhline(y=19, color='k',linewidth=1.5)
ax21.axhline(y=23, color='k',linewidth=3)
ax21.axvline(x=0, color='k',linewidth=3)
ax21.axvline(x=5, color='k',linewidth=3)

plt.savefig("/home/chm/RNA/COMPLETE_PAPER/figures/figure4b.png",dpi=1000)
plt.show()

## Figure S6

In [ ]:
HUB2=hubgenes[hubgenes['Group']=='human_WGCNA_group2']
HUB3=hubgenes[hubgenes['Group']=='human_WGCNA_group3']
HUB6=hubgenes[hubgenes['Group']=='human_WGCNA_group6']

HUB2_605=hubgenes_605[hubgenes_605['Group']=='human_WGCNA_group2']
HUB3_605=hubgenes_605[hubgenes_605['Group']=='human_WGCNA_group3']
HUB6_605=hubgenes_605[hubgenes_605['Group']=='human_WGCNA_group6']

HUB2 = HUB2.rename(columns={'Name':'gene'})
HUB3 = HUB3.rename(columns={'Name':'gene'})
HUB6 = HUB6.rename(columns={'Name':'gene'})

HUB2_605 = HUB2_605.rename(columns={'Name':'gene'})
HUB3_605 = HUB3_605.rename(columns={'Name':'gene'})
HUB6_605 = HUB6_605.rename(columns={'Name':'gene'})

hubgenes = hubgenes.rename(columns={'Name':'gene'})
hubgenes_605 = hubgenes_605.rename(columns={'Name':'gene'})
DEG = pd.concat([DEG_G2,DEG_G3,DEG_G6], axis=0)



DF = DEG_norm_count_mfa.reset_index(drop=False)
DF['gene'] = DF['index'].str.split('_').str[0]

DF_all = DF.iloc[:,1:]
DF_all = DF_all.set_index('gene')
DF_HUB20 = (DF[DF['gene'].isin(hubgenes['gene'].unique())]).iloc[:,1:]
DF_HUB20 = DF_HUB20.set_index('gene')
DF_HUB605 = (DF[DF['gene'].isin(hubgenes_605['gene'].unique())]).iloc[:,1:]
DF_HUB605 = DF_HUB605.set_index('gene')
DF_HUB2 = (DF[DF['gene'].isin(HUB2['gene'].unique())]).iloc[:,1:]
DF_HUB2 = DF_HUB2.set_index('gene')
DF_HUB3 = (DF[DF['gene'].isin(HUB3['gene'].unique())]).iloc[:,1:]
DF_HUB3 = DF_HUB3.set_index('gene')
DF_HUB6 = (DF[DF['gene'].isin(HUB6['gene'].unique())]).iloc[:,1:]
DF_HUB6 = DF_HUB6.set_index('gene')

DF_HUBCC = DF_HUB20[DF_HUB20.index.isin(hubgenes_cancer['gene'].unique())]


PT_1 = (DF[DF['gene'].isin(PT1['gene'].unique())]).iloc[:,1:]
PT_1 = PT_1.set_index('gene')
PT_2 = (DF[DF['gene'].isin(PT2['gene'].unique())]).iloc[:,1:]
PT_2 = PT_2.set_index('gene')
PT_4 = (DF[DF['gene'].isin(PT4['gene'].unique())]).iloc[:,1:]
PT_4 = PT_4.set_index('gene')


DF_all = DF_all.T
DF_HUB20 = DF_HUB20.T
DF_HUBCC = DF_HUBCC.T
DF_HUB605 = DF_HUB605.T
DF_HUB2 = DF_HUB2.T
DF_HUB3 = DF_HUB3.T
DF_HUB6 = DF_HUB6.T
PT_1 = PT_1.T
PT_2 = PT_2.T
PT_4 = PT_4.T

In [ ]:
def adding_age(T):
    T = T.round()
    T = T.astype(int)
    conditionlist = [T.index == 'A_FST',
                     T.index == 'B_FST',
                     T.index == 'A_SND',
                     T.index == 'B_SND',
                     T.index == 'A_TRD',
                     T.index == 'B_TRD',
                 
                     T.index == 'C_FST',
                     T.index == 'D_FST',
                     T.index == 'C_SND',
                     T.index == 'D_SND',
                     T.index == 'C_TRD',
                     T.index == 'D_TRD',
                 
                     T.index == 'E_FST',
                     T.index == 'F_FST',
                     T.index == 'E_SND',
                     T.index == 'F_SND',
                     T.index == 'E_TRD',
                     T.index == 'F_TRD',
                 
                     T.index == 'G_FST',
                     T.index == 'H_FST',
                     T.index == 'G_SND',
                     T.index == 'H_SND',
                     T.index == 'G_TRD',
                     T.index == 'H_TRD']
    choicelist = [3,3,4,4,5,5,7,7,8,8,9,9,12,12,13,13,14,14,16,16,17,17,18,18]
    T['age'] = np.select(conditionlist,choicelist, default='Not Specified')


    conditionlist = [T.index.str.split('_').str[0] == 'A',
                     T.index.str.split('_').str[0] == 'B',
                     T.index.str.split('_').str[0] == 'C',
                     T.index.str.split('_').str[0] == 'D',
                     T.index.str.split('_').str[0] == 'E',
                     T.index.str.split('_').str[0] == 'F',
                     T.index.str.split('_').str[0] == 'G',
                     T.index.str.split('_').str[0] == 'H']
    choicelist = [1,1,2,2,3,3,4,4]
    T['Group'] = np.select(conditionlist,choicelist, default='Not Specified')

    T['age'] = pd.to_numeric(T['age'])
    T['Group'] = pd.to_numeric(T['Group'])
    
    return T

In [ ]:
DF_all = adding_age(DF_all)
DF_HUB20 = adding_age(DF_HUB20)
DF_HUBCC = adding_age(DF_HUBCC)
DF_HUB605 = adding_age(DF_HUB605)
DF_HUB2 = adding_age(DF_HUB2)
DF_HUB3 = adding_age(DF_HUB3)
DF_HUB6 = adding_age(DF_HUB6)
PT_1 = adding_age(PT_1)
PT_2= adding_age(PT_2)
PT_4 = adding_age(PT_4)

DF_all = DF_all.reset_index(drop=False)
DF_HUB20 = DF_HUB20.reset_index(drop=False)
DF_HUBCC = DF_HUBCC.reset_index(drop=False)
DF_HUB605 = DF_HUB605.reset_index(drop=False)
DF_HUB2 = DF_HUB2.reset_index(drop=False)
DF_HUB3 = DF_HUB3.reset_index(drop=False)
DF_HUB6 = DF_HUB6.reset_index(drop=False)
PT_1 = PT_1.reset_index(drop=False)
PT_2 = PT_2.reset_index(drop=False)
PT_4 = PT_4.reset_index(drop=False)

## Figure S7a

In [ ]:
import scikit_posthocs as sp
import matplotlib.gridspec as gridspec
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


plt.figure(figsize = (20,15))
gs = gridspec.GridSpec(nrows=6, ncols=5,height_ratios=[0.1,0.5,0.3,0.1,0.5,0.3], width_ratios=[1,1,1,1,1],hspace=0.5,wspace=0.3)

for n, (DF, DF_name,num) in enumerate(zip([DF_all,PT_1,PT_2,PT_4,DF_HUB2,DF_HUB3,DF_HUB6,DF_HUBCC,DF_HUB20,DF_HUB605],
                                      ['All genes','PT_1','PT_2','PT_4','HUB_PT1','HUB_PT2','HUB_PT4','HUB_Cancer','HUB_20','HUB_All'],
                                      [0,1,2,3,4,0,1,2,3,4])):
 
    if DF_name.startswith('HUB') and DF_name != 'HUB_PT1':
        ax0 = plt.subplot(gs[3,num])
    else:
        ax0 = plt.subplot(gs[0,num])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, -0.8, DF_name, ha='center', va='bottom', size=30)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)

    PCA_DF_X = (DF.iloc[:,1:-2]).values
    PCA_DF_X = StandardScaler().fit_transform(PCA_DF_X)
    PCA_DF_Y = DF.loc[:,['Group']].values

    pca = PCA(random_state=12)
    pca.fit_transform(PCA_DF_X)
    principalComponents = pca.fit_transform(PCA_DF_X)

    percent_variance = np.round(pca.explained_variance_ratio_*100, decimals =2)
    PC_1_= percent_variance[0].astype(str)
    PC_2_= percent_variance[1].astype(str)
    
    principalDf = pd.DataFrame(data = principalComponents)
    finalDataFrame = pd.concat([principalDf, DF[['index','age','Group']]], axis=1)

    #print(finalDataFrame[[0,1,'index','age']])
    
    #Graph
    columns = []
    for i in range(len(percent_variance)):
        columns.append(f'PC{i+1}')
    

    y_max = percent_variance.max()                                                                                            
    y_loc = y_max*0.8
    
    if DF_name.startswith('HUB') and DF_name != 'HUB_PT1':
        ax1 = plt.subplot(gs[5,num])
    else:
        ax1 = plt.subplot(gs[2,num])

    ax1.bar(x = range(len(percent_variance)), height=percent_variance, tick_label=columns)
    plt.xticks(fontsize=10,rotation=90)
    plt.yticks(fontsize=10)
    plt.xlabel('Principal Componenet', fontsize=10)
    plt.ylabel('% of Variance', fontsize=10)

    if DF_name.startswith('HUB') and DF_name != 'HUB_PT1':
        ax2 = plt.subplot(gs[4,num])
    else:
        ax2 = plt.subplot(gs[1,num])

    plt.xlabel('PC 1'+' ('+PC_1_+'%)', fontsize = 15)
    plt.ylabel('PC 2'+' ('+PC_2_+'%)', fontsize = 15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid()

    sns.scatterplot(data=finalDataFrame,x=0, y=1,hue='Group',palette=['#ED8080','#90EE90','#ADD8E6','#EFBBF0'],sizes=300, ax= ax2)
    ax2.get_legend().remove()

plt.savefig("./figures/PCAplot.png",dpi=1000)
plt.show()

## Figure S4e

In [ ]:
DEG_G1['G'] = 'TC_pattern1'
DEG_G2['G'] = 'TC_pattern2'
DEG_G3['G'] = 'TC_pattern3'
DEG_G4['G'] = 'TC_pattern4'
DEG_G5['G'] = 'TC_pattern5'
DEG_G6['G'] = 'TC_pattern6'
DEG_G7['G'] = 'TC_pattern7'

DEG_G1_G2_up['G'] = 'PW_G1-G2_up'
DEG_G1_G2_down['G'] = 'PW_G1-G2_down' 
DEG_G2_G3_up['G'] = 'PW_G2-G3_up' 
DEG_G2_G3_down['G'] = 'PW_G2-G3_down' 
DEG_G3_G4_up['G'] = 'PW_G3-G4_up' 
DEG_G3_G4_down['G'] = 'PW_G3-G4_down'

sel_DF = pd.DataFrame()
for i in [DEG_G1,DEG_G2,DEG_G3,DEG_G4,DEG_G5,DEG_G6,DEG_G7,DEG_G1_G2_up,DEG_G1_G2_down,DEG_G2_G3_up,DEG_G2_G3_down,DEG_G3_G4_up,DEG_G3_G4_down]:
    selected = i[['gene','G']]
    sel_DF = pd.concat([sel_DF,selected],axis=0)

## Figure S4a

In [ ]:
from upsetplot import UpSet, from_indicators, generate_counts


def upsetplot(df):
    
    df = df.dropna(axis=0)
    DA = df[['G','gene']]
    DA = DA.dropna(axis=0)
    DA = DA.set_index('gene')

    all_G = DA['G'].unique().tolist()
    all_G.reverse()
    all_gene = DA.index.unique().tolist()
    
    A = DA[DA['G']==all_G[0]]
    B = DA[DA['G']==all_G[1]]
    C = DA[DA['G']==all_G[2]]
    D = DA[DA['G']==all_G[3]]
    E = DA[DA['G']==all_G[4]]
    F = DA[DA['G']==all_G[5]]
    G = DA[DA['G']==all_G[6]]
    H = DA[DA['G']==all_G[7]]
    I = DA[DA['G']==all_G[8]]
    J = DA[DA['G']==all_G[9]]
    K = DA[DA['G']==all_G[10]]
    L = DA[DA['G']==all_G[11]]
    M = DA[DA['G']==all_G[12]]

    TOT = pd.concat([A,B,C,D,E,F,G,H,I,J,K,L,M],axis=1)
    TOT.columns=all_G

    TOT = TOT.notnull()

    TOT_G = TOT.groupby(all_G).size()
    
    #DEG
    DEG_upset = UpSet(TOT_G,show_counts=True, show_percentages=False,orientation='horizontal',sort_categories_by=None,facecolor='black')
    DEG_upset.style_subsets(present=['TC_pattern2', 'PW_G1-G2_up'],facecolor='#66c2a5')
    DEG_upset.style_subsets(present=['TC_pattern2', 'PW_G2-G3_up'],facecolor='#66c2a5')
    DEG_upset.style_subsets(present=['TC_pattern2', 'PW_G3-G4_up'],facecolor='#66c2a5')
    DEG_upset.style_subsets(present=['TC_pattern3', 'PW_G1-G2_down'],facecolor='#8da0cb')
    DEG_upset.style_subsets(present=['TC_pattern3', 'PW_G2-G3_down'],facecolor='#8da0cb')
    DEG_upset.style_subsets(present=['TC_pattern3', 'PW_G3-G4_down'],facecolor='#8da0cb')
    DEG_upset.style_subsets(present=['TC_pattern6', 'PW_G1-G2_down'],facecolor='#deb887')
    DEG_upset.style_subsets(present=['TC_pattern6', 'PW_G2-G3_down'],facecolor='#deb887')
    DEG_upset.style_subsets(present=['TC_pattern6', 'PW_G3-G4_down'],facecolor='#deb887')

    DEG_upset.plot()
    
    plt.suptitle('DEG_pairwise_timecourse_intersection',x=0.55,y=1,fontsize=20)
    plt.show()

print(upsetplot(sel_DF))

## Figure S1f

In [ ]:
GENE = file_parsing(GeneCPM)
TPT = file_parsing(TrptCPM)

gene_1 = 'ERBB3'
gene_2 = 'ERBB2'
A = GENE[GENE['gene']==gene_1]
B = GENE[GENE['gene']==gene_2]

fig = plt.figure(figsize = (5,3))
sns.lineplot(data=A,x='age',y='value')
sns.lineplot(data=B,x='age',y='value')
plt.ylabel('CPM',fontsize=10)
plt.xlabel('Age',fontsize=10)
plt.show()

## Figure S2f

In [ ]:
#CTSW
gene_1 = 'ENSMFAT00000023903.1'
gene_2 = 'ENSMFAT00000023907.1'

A = TPT[TPT['gene']==gene_1]
B = TPT[TPT['gene']==gene_2]

fig = plt.figure(figsize = (5,3))
sns.lineplot(data=A,x='age',y='value')
sns.lineplot(data=B,x='age',y='value')
plt.ylabel('CPM',fontsize=10)
plt.xlabel('Age',fontsize=10)
plt.show()